In [1]:
# package needed for reading dcm xray files
import sys
!{sys.executable} -m pip install pytorch-lightning

     |████████████████████████████████| 235kB 4.7MB/s 
     |████████████████████████████████| 829kB 17.4MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=d39a9b1ca1b5d14f38d3a96cd3f2761ec4b7133c947fa0039869c3e513a8a340
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [0]:
# download and unzip dataset
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
!unzip hymenoptera_data.zip

In [0]:
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import datasets, transforms, models
import os


In [0]:
# pretrained model
output_classes = 2

model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, output_classes)

In [0]:
class LightningMNISTClassifier(pl.LightningModule):

    def __init__(self):
        super(LightningMNISTClassifier, self).__init__()

        self.model = model_ft

    def forward(self, x):
        batch_size, channels, width, height = x.size()

        x = self.model(x)
      
        # probability distribution over labels
        x = torch.log_softmax(x, dim=1)

        return x

    def cross_entropy_loss(self, logits, labels):
        return F.nll_loss(logits, labels)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)

        logs = {'train_loss': loss}
        return {'loss': loss, 'log': logs}


    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        return {'val_loss': loss}

    def validation_epoch_end(self, outputs):
        # called at the end of the validation epoch
        # outputs is an array with what you returned in validation_step for each batch
        # outputs = [{'loss': batch_0_loss}, {'loss': batch_1_loss}, ..., {'loss': batch_n_loss}] 
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}

    def prepare_data(self):

        data_transforms = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
        }

        data_dir = 'hymenoptera_data'
        image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                                  data_transforms[x])
                          for x in ['train', 'val']}
        self.data_train = image_datasets['train']
        self.data_val = image_datasets['val']

    def train_dataloader(self):
        return DataLoader(self.data_train, batch_size=64, num_workers=3)

    def val_dataloader(self):
        return DataLoader(self.data_val, batch_size=64, num_workers=3)

    # def test_dataloader(self):
        # return DataLoader(self,data_test, batch_size=64)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [0]:
from pytorch_lightning.loggers import TensorBoardLogger

# default logger used by trainer
logger = TensorBoardLogger(
    save_dir=os.getcwd(),
    name='lightning_logs'
)

In [28]:
from pytorch_lightning.callbacks import ModelCheckpoint

# default used by the Trainer
checkpoint_callback = ModelCheckpoint(
    filepath=os.getcwd(),
    save_top_k=True,
    verbose=True,
    monitor='val_loss',
    mode='min',
    prefix=''
)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Checkpoint directory /content exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)


In [29]:

# train
model = LightningMNISTClassifier()

# you can also set this to find best lr :   auto_lr_find=True
# set this while debugging: fast_dev_run=True
# set maximum epochs: max_epochs=1000
# to automatically find the max batch size that can fit gpu set: auto_scale_batch_size='binsearch'
trainer = pl.Trainer(auto_scale_batch_size='binsearch', logger=logger, checkpoint_callback=checkpoint_callback)

trainer.fit(model)

INFO:lightning:GPU available: True, used: False
INFO:lightning:
   | Name                        | Type              | Params
--------------------------------------------------------------
0  | model                       | ResNet            | 11 M  
1  | model.conv1                 | Conv2d            | 9 K   
2  | model.bn1                   | BatchNorm2d       | 128   
3  | model.relu                  | ReLU              | 0     
4  | model.maxpool               | MaxPool2d         | 0     
5  | model.layer1                | Sequential        | 147 K 
6  | model.layer1.0              | BasicBlock        | 73 K  
7  | model.layer1.0.conv1        | Conv2d            | 36 K  
8  | model.layer1.0.bn1          | BatchNorm2d       | 128   
9  | model.layer1.0.relu         | ReLU              | 0     
10 | model.layer1.0.conv2        | Conv2d            | 36 K  
11 | model.layer1.0.bn2          | BatchNorm2d       | 128   
12 | model.layer1.1              | BasicBlock        | 73 K  
13 | 

INFO:lightning:
Epoch 00000: val_loss reached 19.39974 (best 19.39974), saving model to /content/epoch=0_v0.ckpt as top True


INFO:lightning:
Epoch 00001: val_loss reached 16.61771 (best 16.61771), saving model to /content/epoch=1.ckpt as top True


INFO:lightning:
Epoch 00002: val_loss reached 2.24247 (best 2.24247), saving model to /content/epoch=2.ckpt as top True


INFO:lightning:
Epoch 00003: val_loss reached 1.48076 (best 1.48076), saving model to /content/epoch=3.ckpt as top True


...

INFO:lightning:
Epoch 00015: val_loss  was not in top True


INFO:lightning:
Epoch 00016: val_loss  was not in top True


INFO:lightning:
Epoch 00017: val_loss  was not in top True


INFO:lightning:Detected KeyboardInterrupt, attempting graceful shutdown...


1